# IMPORTS

In [ ]:
import csv
import json
import pandas as pd
import numpy as np

import os,shutil, sys
import time
import datetime
from datequarter import DateQuarter

import re
import string
punc = string.punctuation
from ast import literal_eval

In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm
import joblib

In [ ]:
import dask
import dask.dataframe as dd

In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]")\
     .config("spark.executor.memory", "700g")\
     .config("spark.driver.memory", "500g")\
     .config("spark.memory.offHeap.enabled",True)\
     .config("spark.memory.offHeap.size","256g").appName('sachin').getOrCreate()

In [ ]:
from functools import reduce 
from pyspark.sql import DataFrame
from pyspark.sql.functions import sum, col, desc, countDistinct, isnan, when, count, isnull, coalesce, substring
from pyspark.sql.types import *
import pyspark.sql.functions as F

In [ ]:
os.chdir(r"E:\JOBS_DATA\Main\2021")
os.listdir()

['Main_2021-01.txt',
 'Main_2021-02.txt',
 'Main_2021-03.txt',
 'Main_2021-04.txt',
 'Main_2021-05.txt',
 'Main_2021-06.txt',
 'Main_2021-07.txt',
 'Main_2021-08.txt',
 'Main_2021-09.txt',
 'Main_2021-10.txt',
 'Main_2021-11.txt',
 'Main_2021-12.txt']

In [ ]:
os.getcwd()

'E:\\BGT_All\\Main\\2021'

In [8]:
df = spark.read.options(delimiter='\t').csv(r"{0}\\{1}".format(os.getcwd(), 'Main_2021-01.txt'), header=True, inferSchema=True)
df.count(), len(df.columns)

(3206264, 53)

In [10]:
df.groupBy('CanonTitle').count().sort(desc('count')).show()

+--------------------+-------+
|          CanonTitle|  count|
+--------------------+-------+
|                  na|1155897|
|    Registered Nurse|  74651|
|Class A CDL Truck...|  60764|
|     Sales Associate|  27704|
|Licensed Practica...|  26763|
|Customer Service ...|  24362|
|        Merchandiser|  20072|
|     Delivery Driver|  20002|
|    Security Officer|  19881|
|   Assistant Manager|  19482|
|Sales Representative|  19318|
|Certified Nursing...|  18215|
|Administrative As...|  18152|
|Maintenance Techn...|  17988|
|Software Developm...|  17571|
|           Caregiver|  17348|
|Speech Language P...|  15068|
|              Intern|  14553|
|        Receptionist|  13150|
|             Cashier|  12890|
+--------------------+-------+
only showing top 20 rows



In [11]:
df.select(countDistinct("CanonTitle")).show()

+--------------------------+
|count(DISTINCT CanonTitle)|
+--------------------------+
|                      9707|
+--------------------------+



In [13]:
df.groupBy('CleanTitle').count().sort(desc('count')).show()

+--------------------+-----+
|          CleanTitle|count|
+--------------------+-----+
|Customer Service ...|12664|
|Administrative As...| 9513|
|    Registered Nurse| 9487|
|Maintenance Techn...| 8842|
|     Sales Associate| 8127|
|Retail Sales Asso...| 7738|
|     Delivery Driver| 7505|
|   Assistant Manager| 7496|
| Warehouse Associate| 7472|
|Licensed Practica...| 7236|
|Remote Work-Home ...| 7082|
|   Medical Assistant| 6975|
|              Server| 6935|
|Sales Representative| 6425|
|         Housekeeper| 6277|
|             Cashier| 6166|
|          Dishwasher| 6121|
|     Project Manager| 5651|
|     General Manager| 5490|
|                Cook| 5236|
+--------------------+-----+
only showing top 20 rows



In [14]:
df.select(countDistinct("CleanTitle")).show()

+--------------------------+
|count(DISTINCT CleanTitle)|
+--------------------------+
|                   1059581|
+--------------------------+



# Read all files into single rdd

In [18]:
os.chdir('..')
os.listdir()

['2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021']

In [19]:
os.getcwd()

'E:\\BGT_All\\Main'

In [20]:
# dfmain= spark.read.options(delimiter='\t').csv(r"{0}\\*\\*.txt".format(os.getcwd()), header=True, inferSchema=True)
# dfmain.count(), len(dfmain.columns)
#commented since it tales atleast 2 hours, and we need only 3 yrs for now

In [21]:
def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs)

In [22]:
for yr in ['2019', '2020', '2021']:
    os.chdir(yr)
    for f in tqdm(os.listdir()):
        if f.endswith('.txt') and f!='Main_2021-01.txt':
            print(f)
            d1=  spark.read.options(delimiter='\t').csv(r"{0}\\{1}".format(os.getcwd(), f), header=True, inferSchema=True)
            df= unionAll(df, d1)
            print(f, 'done')
            print(df.count(), len(df.columns))
    os.chdir('..')

  0%|          | 0/24 [00:00<?, ?it/s]

Main_2019-01.txt
Main_2019-01.txt done


  4%|▍         | 1/24 [00:09<03:33,  9.27s/it]

6019835 53
Main_2019-02.txt
Main_2019-02.txt done


 12%|█▎        | 3/24 [00:20<02:19,  6.64s/it]

...

107044983 53
Main_2021-11.txt
Main_2021-11.txt done


 92%|█████████▏| 11/12 [11:56<01:15, 75.75s/it]

111109056 53
Main_2021-12.txt
Main_2021-12.txt done


100%|██████████| 12/12 [13:04<00:00, 65.40s/it]

115032642 53


In [23]:
print(df.count(), len(df.columns))

115032642 53


In [26]:
print(df.groupBy('CleanTitle').count().sort(desc('count')).show())
print(df.select(countDistinct("CleanTitle")).show())

+--------------------+------+
|          CleanTitle| count|
+--------------------+------+
|Customer Service ...|460464|
|Administrative As...|389372|
|     Sales Associate|336486|
|    Registered Nurse|301362|
|Maintenance Techn...|272454|
|   Assistant Manager|250389|
|             Cashier|243656|
|              Server|224434|
|     Delivery Driver|223514|
|     Project Manager|219287|
| Warehouse Associate|215569|
|Sales Representative|209158|
|          Dishwasher|203740|
|Retail Sales Asso...|200397|
|         Housekeeper|189604|
|        Receptionist|189548|
|Licensed Practica...|179748|
|   Medical Assistant|178245|
|     General Manager|175204|
|           Caregiver|168901|
+--------------------+------+
only showing top 20 rows

None
+--------------------------+
|count(DISTINCT CleanTitle)|
+--------------------------+
|                  20876953|
+--------------------------+

None


In [27]:
print(df.groupBy('CanonTitle').count().sort(desc('count')).show())
print(df.select(countDistinct("CanonTitle")).show())

+--------------------+--------+
|          CanonTitle|   count|
+--------------------+--------+
|                  na|43105144|
|    Registered Nurse| 2257648|
|Class A CDL Truck...| 1384766|
|     Sales Associate| 1193202|
|Customer Service ...|  908564|
|Software Developm...|  770437|
|Administrative As...|  754870|
|     Delivery Driver|  744995|
|   Assistant Manager|  729838|
|Licensed Practica...|  722791|
|Sales Representative|  695753|
|    Security Officer|  594893|
|Maintenance Techn...|  583212|
|        Merchandiser|  543692|
|           Caregiver|  542082|
|             Cashier|  532219|
|        Receptionist|  513119|
|Certified Nursing...|  464064|
|              Intern|  463089|
|         Housekeeper|  413421|
+--------------------+--------+
only showing top 20 rows

None
+--------------------------+
|count(DISTINCT CanonTitle)|
+--------------------------+
|                     16495|
+--------------------------+

None


In [28]:
print(df.groupBy('OccFam').count().sort(desc('count')).show())
print(df.select(countDistinct("OccFam")).show())

+------+--------+
|OccFam|   count|
+------+--------+
|    29|12845457|
|    11|12319167|
|    43|12123827|
|    41|12087497|
|    15|10818283|
|    53| 7510903|
|    13| 6778965|
|    35| 5540394|
|    na| 4607626|
|    49| 3960876|
|    51| 3421095|
|    25| 3010828|
|    31| 2968941|
|    17| 2935899|
|    39| 2584982|
|    27| 2551205|
|    37| 2357710|
|    47| 1623539|
|    33| 1507395|
|    21| 1455873|
+------+--------+
only showing top 20 rows

None
+----------------------+
|count(DISTINCT OccFam)|
+----------------------+
|                    25|
+----------------------+

None


In [29]:
print(df.groupBy('OccFamName').count().sort(desc('count')).show())
print(df.select(countDistinct("OccFamName")).show())

+--------------------+--------+
|          OccFamName|   count|
+--------------------+--------+
|Healthcare Practi...|12845457|
|Management Occupa...|12319167|
|Office and Admini...|12123827|
|Sales and Related...|12087497|
|Computer and Math...|10818283|
|Transportation an...| 7510903|
|Business and Fina...| 6778965|
|Food Preparation ...| 5540394|
|                  na| 4607626|
|Installation, Mai...| 3960876|
|Production Occupa...| 3421095|
|Education, Traini...| 3010828|
|Healthcare Suppor...| 2968941|
|Architecture and ...| 2935899|
|Personal Care and...| 2584982|
|Arts, Design, Ent...| 2551205|
|Building and Grou...| 2357710|
|Construction and ...| 1623539|
|Protective Servic...| 1507395|
|Community and Soc...| 1455873|
+--------------------+--------+
only showing top 20 rows

None
+--------------------------+
|count(DISTINCT OccFamName)|
+--------------------------+
|                        25|
+--------------------------+

None


In [30]:
df.select(countDistinct("OccFamName")).show()

+--------------------------+
|count(DISTINCT OccFamName)|
+--------------------------+
|                        25|
+--------------------------+



In [31]:
print(df.groupBy('SOC').count().sort(desc('count')).show())
print(df.select(countDistinct("SOC")).show())

+-------+-------+
|    SOC|  count|
+-------+-------+
|29-1141|5140487|
|     na|4607626|
|41-2031|3266127|
|15-1132|3217064|
|41-4012|3091536|
|15-1199|2829279|
|43-4051|2827765|
|53-3032|2756055|
|11-9199|2551227|
|41-1011|2157067|
|53-7062|2118342|
|11-9111|1348204|
|35-3021|1339760|
|43-5081|1328208|
|43-6014|1320849|
|49-9071|1301667|
|11-1021|1102033|
|13-1071|1088607|
|11-9051|1031099|
|11-2022|1010261|
+-------+-------+
only showing top 20 rows

None
+-------------------+
|count(DISTINCT SOC)|
+-------------------+
|                838|
+-------------------+

None


In [32]:
print(df.groupBy('SOCName').count().sort(desc('count')).show())
print(df.select(countDistinct("SOCName")).show())

+--------------------+-------+
|             SOCName|  count|
+--------------------+-------+
|   Registered Nurses|5140487|
|                  na|4607626|
| Retail Salespersons|3266127|
|Software Develope...|3217064|
|Sales Representat...|3091536|
|Computer Occupati...|2829279|
|Customer Service ...|2827765|
|Heavy and Tractor...|2756055|
| Managers, All Other|2551227|
|First-Line Superv...|2157067|
|Laborers and Frei...|2118342|
|Medical and Healt...|1348204|
|Combined Food Pre...|1339760|
|Stock Clerks and ...|1328208|
|Secretaries and A...|1320849|
|Maintenance and R...|1301667|
|General and Opera...|1102033|
|Human Resources S...|1088607|
|Food Service Mana...|1031099|
|      Sales Managers|1010261|
+--------------------+-------+
only showing top 20 rows

None
+-----------------------+
|count(DISTINCT SOCName)|
+-----------------------+
|                    838|
+-----------------------+

None


In [33]:
print(df.groupBy('ONET').count().sort(desc('count')).show())
print(df.select(countDistinct("ONET")).show())

+----------+-------+
|      ONET|  count|
+----------+-------+
|        na|4607626|
|29-1141.00|4479535|
|41-2031.00|3266127|
|15-1132.00|3217064|
|41-4012.00|3091536|
|53-3032.00|2756055|
|43-4051.00|2425658|
|11-9199.00|2297047|
|41-1011.00|2157067|
|53-7062.00|2118342|
|11-9111.00|1348204|
|35-3021.00|1339760|
|43-6014.00|1320849|
|49-9071.00|1301667|
|11-1021.00|1102033|
|13-1071.00|1088607|
|11-9051.00|1031099|
|11-2022.00|1010261|
|37-2011.00|1002101|
|33-9032.00| 963292|
+----------+-------+
only showing top 20 rows

None
+--------------------+
|count(DISTINCT ONET)|
+--------------------+
|                1063|
+--------------------+

None


In [35]:
print(df.groupBy('ONETName').count().sort(desc('count')).show())
print(df.select(countDistinct("ONETName")).show())

+--------------------+-------+
|            ONETName|  count|
+--------------------+-------+
|                  na|4607626|
|   Registered Nurses|4479535|
| Retail Salespersons|3266127|
|Software Develope...|3217064|
|Sales Representat...|3091536|
|Heavy and Tractor...|2756055|
|Customer Service ...|2425658|
| Managers, All Other|2297047|
|First-Line Superv...|2157067|
|Laborers and Frei...|2118342|
|Medical and Healt...|1348204|
|Combined Food Pre...|1339760|
|Secretaries and A...|1320849|
|Maintenance and R...|1301667|
|General and Opera...|1102033|
|Human Resources S...|1088607|
|Food Service Mana...|1031099|
|      Sales Managers|1010261|
|Janitors and Clea...|1002101|
|     Security Guards| 963292|
+--------------------+-------+
only showing top 20 rows

None
+------------------------+
|count(DISTINCT ONETName)|
+------------------------+
|                    1063|
+------------------------+

None


In [40]:
print(df.groupBy('Sector').count().sort(desc('count')).show())
print(df.select(countDistinct("Sector")).show())

+------+--------+
|Sector|   count|
+------+--------+
|    na|27220460|
|    62|19622837|
| 44-45|10860796|
|    72| 7731641|
|    54| 7587924|
|    52| 7279322|
| 31-33| 7188292|
|    61| 4682098|
|    56| 4682048|
| 48-49| 4276919|
|    92| 2916362|
|    51| 2679181|
|    53| 2055314|
|    81| 1931030|
|    23| 1755801|
|    71|  862789|
|    42|  685346|
|    22|  380398|
|    21|  273047|
|    11|  181239|
+------+--------+
only showing top 20 rows

None
+----------------------+
|count(DISTINCT Sector)|
+----------------------+
|                    22|
+----------------------+

None


In [68]:
print(df.groupBy('SectorName').count().sort(desc('count')).show())
print(df.select(countDistinct("SectorName")).show())

+--------------------+--------+
|          SectorName|   count|
+--------------------+--------+
|                  na|27220460|
|Health Care and S...|19622837|
|        Retail Trade|10860796|
|Accommodation and...| 7731641|
|Professional, Sci...| 7587924|
|Finance and Insur...| 7279322|
|       Manufacturing| 7188292|
|Educational Services| 4682098|
|Administrative an...| 4682048|
|Transportation an...| 4276919|
|Public Administra...| 2916362|
|         Information| 2679181|
|Real Estate and R...| 2055314|
|Other Services (e...| 1931030|
|        Construction| 1755801|
|Arts, Entertainme...|  862789|
|     Wholesale Trade|  685346|
|           Utilities|  380398|
|Mining, Quarrying...|  273047|
|Agriculture, Fore...|  181239|
+--------------------+--------+
only showing top 20 rows

None
+--------------------------+
|count(DISTINCT SectorName)|
+--------------------------+
|                        22|
+--------------------------+

None


In [54]:
print(df.groupBy('Exp').count().sort(desc('count')).show())
print(df.select(countDistinct("Exp")).show())

+------+--------+
|   Exp|   count|
+------+--------+
|-999.0|62645689|
|   1.0|12742418|
|   2.0|10861277|
|   3.0| 8164806|
|   5.0| 7745231|
|   4.0| 2686535|
|  10.0| 2166148|
|   0.5| 2141325|
|   7.0| 1518262|
|   8.0| 1402642|
|   6.0| 1096059|
|  15.0|  384068|
|  0.25|  359150|
|  12.0|  332491|
|  0.08|  270878|
|   9.0|  165748|
|   1.5|   95291|
|  14.0|   54724|
|  11.0|   48985|
|  13.0|   41239|
+------+--------+
only showing top 20 rows

None
+-------------------+
|count(DISTINCT Exp)|
+-------------------+
|                 49|
+-------------------+

None


In [58]:
print(df.groupBy('Employer').count().sort(desc('count')).show())
print(df.select(countDistinct("Employer")).show())

+--------------------+--------+
|            Employer|   count|
+--------------------+--------+
|                  na|21284394|
|              Amazon| 1162575|
|   Anthem Blue Cross|  607580|
|Lowe's Companies,...|  530288|
|Marriott Internat...|  342434|
|     Walmart / Sam's|  316357|
|            Deloitte|  303811|
|The Home Depot In...|  284424|
|Hospital Corporat...|  269647|
|    Allied Universal|  253425|
|      Dollar General|  245264|
|          CVS Health|  240223|
|         Wells Fargo|  231169|
|Walgreens Boots A...|  218070|
|               FedEx|  204491|
|              Humana|  202550|
|  UnitedHealth Group|  184088|
|             Aramark|  170884|
|United Parcel Ser...|  165432|
|          McDonald's|  164876|
+--------------------+--------+
only showing top 20 rows

None
+------------------------+
|count(DISTINCT Employer)|
+------------------------+
|                 2731477|
+------------------------+

None


In [60]:
print(df.groupBy('State').count().sort(desc('count')).show(63))
print(df.select(countDistinct("State")).show())

+--------------------+--------+
|               State|   count|
+--------------------+--------+
|          California|15634984|
|               Texas| 9920739|
|             Florida| 7027715|
|            New York| 5210700|
|            Illinois| 4603799|
|                Ohio| 4130345|
|        Pennsylvania| 3958575|
|      North Carolina| 3910673|
|             Georgia| 3739303|
|            Virginia| 3401356|
|       Massachusetts| 3379215|
|            Colorado| 3354434|
|            Michigan| 3227732|
|             Arizona| 3198375|
|          Washington| 3009193|
|          New Jersey| 2890563|
|             Indiana| 2371423|
|           Minnesota| 2365942|
|           Tennessee| 2297249|
|           Wisconsin| 2292401|
|            Maryland| 2094309|
|            Missouri| 1984448|
|              Oregon| 1596207|
|             Alabama| 1401215|
|         Connecticut| 1283587|
|            Kentucky| 1265068|
|      South Carolina| 1225993|
|            Oklahoma| 1198178|
|       

In [61]:
print(df.groupBy('City').count().sort(desc('count')).show())
print(df.select(countDistinct("City")).show())

+-------------+-------+
|         City|  count|
+-------------+-------+
|     New York|2560790|
|  Los Angeles|1581424|
|           na|1385582|
|      Houston|1374886|
|      Chicago|1358005|
|      Atlanta|1150151|
|       Dallas|1138379|
|San Francisco|1098632|
|    San Diego|1014712|
|      Phoenix|1003558|
|       Austin| 974795|
|       Denver| 922327|
|   Washington| 920587|
|      Seattle| 895281|
|    Charlotte| 867340|
|       Boston| 840501|
|     Columbus| 772708|
| Philadelphia| 772115|
|  San Antonio| 766528|
|        Miami| 742562|
+-------------+-------+
only showing top 20 rows

None
+--------------------+
|count(DISTINCT City)|
+--------------------+
|               19710|
+--------------------+

None


In [66]:
print(df.groupBy('Lat').count().sort(desc('count')).show())
print(df.select(countDistinct("Lat")).show())

+-------+-------+
|    Lat|  count|
+-------+-------+
|40.7501|1988625|
| -999.0|1375523|
|41.8746|1315511|
|29.7194|1262870|
|33.9746|1216865|
|33.6488|1105342|
|37.7798|1073819|
| 32.787|1068999|
|33.4507| 960177|
|32.7211| 935406|
|39.7506| 866033|
|30.2202| 841560|
|35.1943| 836691|
|38.9086| 826931|
|42.3575| 765128|
|47.7593| 756087|
|29.3423| 697473|
|40.1169| 645689|
|39.7751| 635586|
|27.9827| 629369|
+-------+-------+
only showing top 20 rows

None
+-------------------+
|count(DISTINCT Lat)|
+-------------------+
|              44162|
+-------------------+

None


In [67]:
print(df.groupBy('PayFrequency').count().sort(desc('count')).show())
print(df.select(countDistinct("PayFrequency")).show())

+------------+--------+
|PayFrequency|   count|
+------------+--------+
|          na|77202049|
|      hourly|19242267|
|      annual|16397165|
|     monthly| 1165947|
|       daily|  801314|
|      weekly|  223895|
|        none|       4|
|        null|       1|
+------------+--------+

None
+----------------------------+
|count(DISTINCT PayFrequency)|
+----------------------------+
|                           7|
+----------------------------+

None


In [69]:
print(df.groupBy('MaxSalary').count().sort(desc('count')).show())
print(df.select(countDistinct("MaxSalary")).show())

+---------+--------+
|MaxSalary|   count|
+---------+--------+
|   -999.0|77202049|
|  31200.0| 1988114|
|  41600.0| 1121811|
|  33280.0| 1104667|
|  37440.0| 1065406|
|  29120.0|  969017|
|  35360.0|  876623|
|  52000.0|  858165|
|  24960.0|  825092|
|  27040.0|  804286|
| 100000.0|  605028|
|  50000.0|  485335|
|  60000.0|  481180|
|  45760.0|  462276|
|  20800.0|  424327|
|  22880.0|  423750|
| 120000.0|  420692|
|  39520.0|  408847|
|  62400.0|  396823|
|  65000.0|  361620|
+---------+--------+
only showing top 20 rows

None
+-------------------------+
|count(DISTINCT MaxSalary)|
+-------------------------+
|                   206947|
+-------------------------+

None


In [70]:
print(df.groupBy('MaxHrlySalary').count().sort(desc('count')).show())
print(df.select(countDistinct("MaxHrlySalary")).show())

+-------------+--------+
|MaxHrlySalary|   count|
+-------------+--------+
|       -999.0|75052255|
|      -999.00| 2149794|
|         15.0| 1915445|
|         20.0| 1086923|
|         16.0| 1063430|
|         18.0| 1028959|
|         14.0|  932386|
|         17.0|  846829|
|         25.0|  831804|
|         12.0|  793179|
|         13.0|  773534|
|        48.08|  606011|
|        24.04|  488618|
|        28.85|  484583|
|         22.0|  448456|
|        57.69|  421321|
|         10.0|  408181|
|         11.0|  407580|
|         19.0|  395409|
|         30.0|  384745|
+-------------+--------+
only showing top 20 rows

None
+-----------------------------+
|count(DISTINCT MaxHrlySalary)|
+-----------------------------+
|                        16288|
+-----------------------------+

None


In [71]:
print(df.groupBy('SalaryType').count().sort(desc('count')).show())
print(df.select(countDistinct("SalaryType")).show())

+------------------+--------+
|        SalaryType|   count|
+------------------+--------+
|                na|77202053|
|              none|19117047|
|           basepay|16892638|
|             bonus| 1579147|
|        commission|  236913|
|      shiftpremium|    4754|
|shorttermincentive|      89|
|              null|       1|
+------------------+--------+

None
+--------------------------+
|count(DISTINCT SalaryType)|
+--------------------------+
|                         7|
+--------------------------+

None


In [72]:
print(df.groupBy('JobHours').count().sort(desc('count')).show())
print(df.select(countDistinct("JobHours")).show())

+--------+--------+
|JobHours|   count|
+--------+--------+
|fulltime|52040631|
|      na|51653513|
|parttime|11338497|
|    null|       1|
+--------+--------+

None
+------------------------+
|count(DISTINCT JobHours)|
+------------------------+
|                       3|
+------------------------+

None


In [73]:
print(df.groupBy('Internship').count().sort(desc('count')).show())
print(df.select(countDistinct("Internship")).show())

+----------+---------+
|Internship|    count|
+----------+---------+
|         0|113907545|
|         1|  1125092|
|      null|        5|
+----------+---------+

None
+--------------------------+
|count(DISTINCT Internship)|
+--------------------------+
|                         2|
+--------------------------+

None


In [76]:
print(df.groupBy('Edu').count().sort(desc('count')).show())
print(df.select(countDistinct("Edu")).show())

+----+--------+
| Edu|   count|
+----+--------+
|   0|53465407|
|  12|27191182|
|  16|25168094|
|  14| 5566809|
|  18| 2480562|
|  21| 1098780|
|-999|   61807|
|null|       1|
+----+--------+

None
+-------------------+
|count(DISTINCT Edu)|
+-------------------+
|                  7|
+-------------------+

None


In [77]:
print(df.groupBy('MaxEdu').count().sort(desc('count')).show())
print(df.select(countDistinct("MaxEdu")).show())

+----------+--------+
|    MaxEdu|   count|
+----------+--------+
|         0|93917142|
|        16| 7676629|
|        18| 5173027|
|        12| 3998086|
|        14| 2675244|
|        21| 1590856|
|      -999|    1653|
|Bachelor's|       4|
|      null|       1|
+----------+--------+

None
+----------------------+
|count(DISTINCT MaxEdu)|
+----------------------+
|                     8|
+----------------------+

None


In [122]:
d1= df.dropna()
print(df.count(), len(df.columns))
print(d1.count(), len(d1.columns))

115032642 53
114994836 53


In [123]:
d1= d1.filter(d1.Internship=='0')
print(d1.count(), len(d1.columns))

113870020 53


In [81]:
print(d1.groupBy('Internship').count().sort(desc('count')).show())

+----------+---------+
|Internship|    count|
+----------+---------+
|         0|113870020|
+----------+---------+

None


In [124]:
d1= d1.filter(d1.JobHours=='fulltime')
print(d1.count(), len(d1.columns))

51679527 53


In [125]:
d1= d1.filter(d1.PayFrequency!='na')
print(d1.count(), len(d1.columns))

22807848 53


In [84]:
print(d1.groupBy('PayFrequency').count().sort(desc('count')).show())

+------------+--------+
|PayFrequency|   count|
+------------+--------+
|      hourly|12208686|
|      annual| 9426702|
|     monthly|  618252|
|       daily|  410078|
|      weekly|  144130|
+------------+--------+

None


In [126]:
d1= d1.filter(d1.City!='na')
print(d1.count(), len(d1.columns))

22651498 53


In [127]:
d1= d1.filter(d1.Employer!='na')
print(d1.count(), len(d1.columns))

17651227 53


In [87]:
print(d1.groupBy('Employer').count().sort(desc('count')).show())

+--------------------+------+
|            Employer| count|
+--------------------+------+
|              Amazon|513924|
|       US Government| 86568|
|Department of Vet...| 64852|
|               FedEx| 38873|
|             US Army| 38096|
|United Parcel Ser...| 37020|
|State Farm Insura...| 36277|
|    Allied Universal| 31866|
|   Anthem Blue Cross| 31461|
|     Walmart / Sam's| 30336|
|Lowe's Companies,...| 28935|
|            Goodwill| 27253|
|      Domino's Pizza| 26334|
|Marriott Internat...| 26119|
|        Hudson Group| 25466|
|  UnitedHealth Group| 23585|
|           Uc Health| 22926|
|Hilton Hotel Corp...| 21970|
|           Securitas| 21191|
|         Chick-fil-A| 21179|
+--------------------+------+
only showing top 20 rows

None


In [88]:
print(d1.select(countDistinct("Employer")).show())

+------------------------+
|count(DISTINCT Employer)|
+------------------------+
|                 1732984|
+------------------------+

None


In [128]:
d1= d1.filter(d1.Exp!='-999.0')
print(d1.count(), len(d1.columns))

7153311 53


In [129]:
d1= d1.filter(d1.OccFam!='na')
print(d1.count(), len(d1.columns))

6949402 53


In [130]:
d1= d1.filter(d1.SectorName!='na')
print(d1.count(), len(d1.columns))

5238189 53


In [131]:
d1= d1.filter(d1.Specialty!='na')
print(d1.count(), len(d1.columns))

5079912 53


In [132]:
d1= d1.filter(d1.ONETName!='na')
print(d1.count(), len(d1.columns))

5079912 53


In [94]:
print(d1.groupBy('ONETName').count().sort(desc('count')).show())
print(d1.select(countDistinct("ONETName")).show())

+--------------------+------+
|            ONETName| count|
+--------------------+------+
|   Registered Nurses|242078|
|Heavy and Tractor...|218544|
|Sales Representat...|180583|
|Customer Service ...|136480|
|First-Line Superv...|119909|
| Retail Salespersons| 93873|
|Secretaries and A...| 90400|
|Food Service Mana...| 90053|
|Maintenance and R...| 83235|
|Medical and Healt...| 68853|
| Managers, All Other| 62339|
|Human Resources S...| 57866|
|General and Opera...| 56515|
|     Security Guards| 54004|
|Software Develope...| 52364|
| Medical Secretaries| 51120|
|Waiters and Waitr...| 50154|
|  Medical Assistants| 48689|
|Bookkeeping, Acco...| 48280|
|Laborers and Frei...| 47437|
+--------------------+------+
only showing top 20 rows

None
+------------------------+
|count(DISTINCT ONETName)|
+------------------------+
|                     948|
+------------------------+

None


In [112]:
print(d1.groupBy('ONETName').count().sort(desc('count')).show())
print(d1.select(countDistinct("ONETName")).show())

+--------------------+------+
|            ONETName| count|
+--------------------+------+
|   Registered Nurses|212950|
|Heavy and Tractor...|202970|
|Sales Representat...|147995|
|Customer Service ...| 99198|
|First-Line Superv...| 90457|
| Retail Salespersons| 74851|
|Secretaries and A...| 73293|
|Maintenance and R...| 67070|
|Food Service Mana...| 56825|
|     Security Guards| 48192|
|Human Resources S...| 45417|
|Medical and Healt...| 40539|
|Licensed Practica...| 39607|
| Medical Secretaries| 39140|
| Personal Care Aides| 38277|
|Laborers and Frei...| 37014|
|Bookkeeping, Acco...| 36234|
|Software Develope...| 35471|
|      Sales Managers| 35425|
|   Cooks, Restaurant| 34679|
+--------------------+------+
only showing top 20 rows

None
+------------------------+
|count(DISTINCT ONETName)|
+------------------------+
|                     860|
+------------------------+

None


In [133]:
d1= d1.filter(d1.SOCName!='na')
print(d1.count(), len(d1.columns))

5079912 53


In [96]:
print(d1.groupBy('SOCName').count().sort(desc('count')).show())
print(d1.select(countDistinct("SOCName")).show())

+--------------------+------+
|             SOCName| count|
+--------------------+------+
|   Registered Nurses|273807|
|Heavy and Tractor...|218544|
|Sales Representat...|180583|
|Customer Service ...|159503|
|First-Line Superv...|119909|
| Retail Salespersons| 93873|
|Secretaries and A...| 90400|
|Food Service Mana...| 90053|
|Maintenance and R...| 83235|
| Managers, All Other| 75755|
|Computer Occupati...| 73864|
|Medical and Healt...| 68853|
|Human Resources S...| 57866|
|General and Opera...| 56515|
|     Security Guards| 54004|
|Software Develope...| 52364|
| Medical Secretaries| 51120|
|Waiters and Waitr...| 50154|
|  Medical Assistants| 48689|
|Bookkeeping, Acco...| 48280|
+--------------------+------+
only showing top 20 rows

None
+-----------------------+
|count(DISTINCT SOCName)|
+-----------------------+
|                    753|
+-----------------------+

None


In [100]:
print(df.groupBy('Employer').count().sort(desc('count')).show())
print(df.select(countDistinct("Employer")).show())

+--------------------+--------+
|            Employer|   count|
+--------------------+--------+
|                  na|21284394|
|              Amazon| 1162575|
|   Anthem Blue Cross|  607580|
|Lowe's Companies,...|  530288|
|Marriott Internat...|  342434|
|     Walmart / Sam's|  316357|
|            Deloitte|  303811|
|The Home Depot In...|  284424|
|Hospital Corporat...|  269647|
|    Allied Universal|  253425|
|      Dollar General|  245264|
|          CVS Health|  240223|
|         Wells Fargo|  231169|
|Walgreens Boots A...|  218070|
|               FedEx|  204491|
|              Humana|  202550|
|  UnitedHealth Group|  184088|
|             Aramark|  170884|
|United Parcel Ser...|  165432|
|          McDonald's|  164876|
+--------------------+--------+
only showing top 20 rows

None
+------------------------+
|count(DISTINCT Employer)|
+------------------------+
|                 2731477|
+------------------------+

None


In [97]:
print(d1.groupBy('Employer').count().sort(desc('count')).show())
print(d1.select(countDistinct("Employer")).show())

+--------------------+-----+
|            Employer|count|
+--------------------+-----+
|       US Government|59862|
|Department of Vet...|45783|
|   Anthem Blue Cross|25950|
|Lowe's Companies,...|23393|
|    Allied Universal|19412|
|  UnitedHealth Group|18964|
|      Aya Healthcare|17429|
|         Wells Fargo|17414|
|           Uc Health|15292|
|Ryder System Inco...|14760|
|Walgreens Boots A...|12411|
|            Goodwill|12012|
|        Nomad Health|11382|
|             US Army|11141|
|                USAA| 9949|
|Mayo Foundation f...| 9910|
|             Pearson| 9860|
|              Amazon| 8957|
|               FedEx| 8660|
|Hilton Hotel Corp...| 7944|
+--------------------+-----+
only showing top 20 rows

None
+------------------------+
|count(DISTINCT Employer)|
+------------------------+
|                  637626|
+------------------------+

None


In [98]:
print(d1.groupBy('CleanTitle').count().sort(desc('count')).show())
print(d1.select(countDistinct("CleanTitle")).show())

+--------------------+-----+
|          CleanTitle|count|
+--------------------+-----+
|Customer Service ...|32203|
|Sales Representative|22391|
|Maintenance Techn...|21852|
|    Registered Nurse|18339|
|Administrative As...|17695|
|     General Manager|17007|
|   Assistant Manager|15762|
|   Medical Assistant|15439|
|       Store Manager|14814|
|Outside Sales Rep...|13763|
|Assistant Store M...|13180|
|     Sales Associate|11882|
|              Server|10920|
|Inside Sales Repr...|10158|
|     Project Manager|10018|
|    Dental Assistant| 9296|
|           Caregiver| 8548|
|Retail Sales Asso...| 8016|
|Licensed Practica...| 7916|
| Pharmacy Technician| 7795|
+--------------------+-----+
only showing top 20 rows

None
+--------------------------+
|count(DISTINCT CleanTitle)|
+--------------------------+
|                   1461350|
+--------------------------+

None


In [99]:
print(d1.groupBy('CanonTitle').count().sort(desc('count')).show())
print(d1.select(countDistinct("CanonTitle")).show())

+--------------------+-------+
|          CanonTitle|  count|
+--------------------+-------+
|                  na|1519161|
|Class A CDL Truck...| 131647|
|    Registered Nurse| 116841|
|Customer Service ...|  68394|
|Sales Representative|  61949|
|Maintenance Techn...|  46919|
|Administrative As...|  44883|
|   Assistant Manager|  42919|
|     Sales Associate|  35903|
|    Security Officer|  35832|
|           Caregiver|  34574|
|Licensed Practica...|  31412|
|     Delivery Driver|  30588|
|       Store Manager|  26969|
|        Receptionist|  23856|
|   Forklift Operator|  22485|
|           Bartender|  19571|
|Outside Sales Rep...|  19447|
|Assistant Store M...|  18533|
|           Line Cook|  18376|
+--------------------+-------+
only showing top 20 rows

None
+--------------------------+
|count(DISTINCT CanonTitle)|
+--------------------------+
|                      8515|
+--------------------------+

None


In [134]:
d1= d1.filter(d1.CleanTitle!='na')
print(d1.count(), len(d1.columns))

5079816 53


In [135]:
d1= d1.filter(d1.CanonTitle!='na')     
print(d1.count(), len(d1.columns))

3560751 53


In [ ]:
# df.withColumn('CanonTitle', coalesce('CanonTitle', 'CleanTitle')).show()

In [105]:
print(d1.groupBy('SalaryType').count().sort(desc('count')).show())
print(d1.select(countDistinct("SalaryType")).show())

+------------------+-------+
|        SalaryType|  count|
+------------------+-------+
|              none|1677931|
|           basepay|1672033|
|             bonus| 189705|
|        commission|  20810|
|      shiftpremium|    256|
|shorttermincentive|     16|
+------------------+-------+

None
+--------------------------+
|count(DISTINCT SalaryType)|
+--------------------------+
|                         6|
+--------------------------+

None


In [116]:
d1.filter(col("CanonTitle").contains("Software")).groupBy('CanonTitle').count().sort(desc('count')).show()

+--------------------+-----+
|          CanonTitle|count|
+--------------------+-----+
|Software Developm...| 9859|
|  Software Developer| 3108|
|    Software Manager|  679|
|Principal Softwar...|  571|
|Software Engineer...|  514|
|Software Developm...|  412|
|  Software Architect|  400|
|Software Quality ...|  158|
|Software Test Eng...|  148|
|     Software Tester|  135|
|Software Systems ...|  119|
|Junior Software D...|   66|
|Junior Software E...|   55|
|Software Design E...|   48|
|C++ Software Engi...|   39|
|Software Developm...|   36|
|Software Account ...|   36|
|Software Support ...|   27|
|Web Software Deve...|   26|
|Embedded Software...|   10|
+--------------------+-----+
only showing top 20 rows



In [118]:
d1.filter(col("CanonTitle").contains("Data")).groupBy('CanonTitle').count().sort(desc('count')).show()

+--------------------+-----+
|          CanonTitle|count|
+--------------------+-----+
|        Data Analyst| 5560|
|Database Administ...| 1590|
|    Data Entry Clerk| 1466|
|      Data Architect|  978|
|Business Data Ana...|  482|
|Clinical Data Man...|  474|
|Data Entry Techni...|  383|
|Sql Server Databa...|  318|
|    Database Analyst|  298|
|Oracle Database A...|  273|
|Healthcare Data A...|  264|
|  Database Developer|  245|
| Database Specialist|  237|
|   Database Engineer|  229|
|SQL Database Admi...|  179|
|Data Quality Analyst|  170|
|        Data Modeler|  169|
|Data Warehouse De...|  117|
|Data Warehouse An...|   92|
|Marketing Data An...|   85|
+--------------------+-----+
only showing top 20 rows



In [117]:
d1.filter(col("CanonTitle").contains("Software")).groupBy('CleanTitle').count().sort(desc('count')).show()

+--------------------+-----+
|          CleanTitle|count|
+--------------------+-----+
|   Software Engineer|  983|
|Senior Software E...|  812|
|  Software Developer|  757|
|Senior Software D...|  278|
|Senior Software E...|  219|
|Principal Softwar...|  147|
|Software Engineer II|  136|
|Software Engineer...|  132|
|Lead Software Eng...|  128|
|Software Developm...|  124|
|Senior Software E...|  121|
|Embedded Software...|   94|
|Software Engineer...|   88|
|Software Developm...|   87|
|  Software Architect|   74|
|Principal Softwar...|   73|
|Senior Software E...|   66|
|Manager, Software...|   64|
| Software Engineer I|   62|
|Software Project ...|   59|
+--------------------+-----+
only showing top 20 rows



In [136]:
d1= d1.filter(d1.SalaryType=='basepay')     
print(d1.count(), len(d1.columns))

1672033 53


In [137]:
d1.filter(col("CanonTitle").contains("Software")).groupBy('CanonTitle').count().sort(desc('count')).show()

+--------------------+-----+
|          CanonTitle|count|
+--------------------+-----+
|Software Developm...| 4395|
|  Software Developer| 1472|
|Principal Softwar...|  228|
|    Software Manager|  210|
|Software Developm...|  154|
|  Software Architect|  149|
|Software Engineer...|  143|
|Software Quality ...|   74|
|Software Test Eng...|   66|
|Software Systems ...|   60|
|     Software Tester|   52|
|Junior Software E...|   28|
|Junior Software D...|   27|
|Software Design E...|   25|
|Web Software Deve...|   18|
|Software Support ...|   16|
|C++ Software Engi...|   14|
|Software Account ...|   11|
|Software Developm...|   10|
|Embedded Software...|    3|
+--------------------+-----+
only showing top 20 rows



In [142]:
d1.filter(col("CanonTitle").contains("Data")).groupBy('CanonTitle').count().sort(desc('count')).show()

+--------------------+-----+
|          CanonTitle|count|
+--------------------+-----+
|        Data Analyst| 2284|
|Database Administ...|  792|
|    Data Entry Clerk|  706|
|      Data Architect|  396|
|Business Data Ana...|  179|
|Data Entry Techni...|  153|
|Clinical Data Man...|  143|
|Oracle Database A...|  142|
|    Database Analyst|  127|
| Database Specialist|  117|
|Sql Server Databa...|  108|
|   Database Engineer|   85|
|  Database Developer|   80|
|Data Quality Analyst|   73|
|Healthcare Data A...|   70|
|        Data Modeler|   69|
|SQL Database Admi...|   64|
|Data Warehouse De...|   41|
|Data Warehouse An...|   40|
|Business Intellig...|   36|
+--------------------+-----+
only showing top 20 rows



In [141]:
print(d1.groupBy('Employer').count().sort(desc('count')).show())
print(d1.select(countDistinct("Employer")).show())

+--------------------+-----+
|            Employer|count|
+--------------------+-----+
|       US Government|34200|
|Department of Vet...|32457|
|    Allied Universal|11679|
|Lowe's Companies,...| 7303|
|             US Army| 7287|
|Ryder System Inco...| 6076|
|Walgreens Boots A...| 6023|
|         Wells Fargo| 5509|
|        Nomad Health| 5410|
|                USAA| 5005|
|                 G4S| 4431|
|Washington Univer...| 4311|
|           Uc Health| 4024|
|            Goodwill| 3986|
|colorado state go...| 3838|
|               Honor| 3768|
|University of Was...| 3584|
|Internal Revenue ...| 3561|
|     Department Army| 3512|
|Nursefly Travel N...| 3407|
+--------------------+-----+
only showing top 20 rows

None
+------------------------+
|count(DISTINCT Employer)|
+------------------------+
|                  313773|
+------------------------+

None


In [152]:
d1.filter(~((d1.Employer.contains('Government')) |
 (d1.Employer.contains('government')) |
 (d1.Employer.contains('Department ')) |
 (d1.Employer.contains('department ')) |
 (d1.Employer.contains('Army')) |
 (d1.Employer.contains('army')))
 ).groupBy('Employer').count().sort(desc('count')).show()

+--------------------+-----+
|            Employer|count|
+--------------------+-----+
|    Allied Universal|11679|
|Lowe's Companies,...| 7303|
|Ryder System Inco...| 6076|
|Walgreens Boots A...| 6023|
|         Wells Fargo| 5507|
|        Nomad Health| 5410|
|                USAA| 5005|
|                 G4S| 4431|
|Washington Univer...| 4311|
|           Uc Health| 4024|
|            Goodwill| 3985|
|               Honor| 3768|
|University of Was...| 3584|
|Internal Revenue ...| 3561|
|Nursefly Travel N...| 3407|
|           Pizza Hut| 3371|
|                Aldi| 3279|
|Texas Health Reso...| 3254|
|         Holiday Inn| 3174|
|Indian Health Ser...| 3149|
+--------------------+-----+
only showing top 20 rows



In [138]:
print(d1.groupBy('Edu').count().sort(desc('count')).show())
print(d1.select(countDistinct("Edu")).show())

+----+------+
| Edu| count|
+----+------+
|  12|592844|
|   0|557402|
|  16|329360|
|  14|125506|
|  18| 43183|
|  21| 21596|
|-999|  2142|
+----+------+

None
+-------------------+
|count(DISTINCT Edu)|
+-------------------+
|                  7|
+-------------------+

None


In [139]:
print(d1.groupBy('Degree').count().sort(desc('count')).show())
print(d1.select(countDistinct("Degree")).show())

+-----------+------+
|     Degree| count|
+-----------+------+
|High School|592844|
|         na|560587|
| Bachelor's|329360|
|Associate's|124463|
|   Master's| 43183|
|        PhD| 21596|
+-----------+------+

None
+----------------------+
|count(DISTINCT Degree)|
+----------------------+
|                     6|
+----------------------+

None


In [140]:
print(d1.groupBy('Exp').count().sort(desc('count')).show())
print(d1.select(countDistinct("Exp")).show())

+----+------+
| Exp| count|
+----+------+
| 1.0|619569|
| 2.0|375305|
| 3.0|245939|
| 5.0|158927|
| 4.0| 72440|
| 0.5| 70005|
|10.0| 34448|
| 6.0| 18934|
| 7.0| 18553|
| 8.0| 13997|
|0.25| 13129|
|0.08|  5747|
| 1.5|  5189|
|12.0|  5085|
|15.0|  4750|
| 9.0|  2451|
|0.42|  1664|
|0.33|  1018|
|11.0|   925|
|14.0|   914|
+----+------+
only showing top 20 rows

None
+-------------------+
|count(DISTINCT Exp)|
+-------------------+
|                 38|
+-------------------+

None


In [143]:
d1= d1.filter(d1.Edu!='-999')
print(d1.count(), len(d1.columns))

1669891 53


In [146]:
columns= [
 'JobID',
 'JobDate',
 'CleanTitle',
 'CanonTitle',
 'OccFam',
 'OccFamName',
 'SOC',
 'SOCName',
 'ONET',
 'ONETName',
 'Employer',
 'Sector',
 'SectorName',
 'NAICS3',
 'City',
 'County',
 'State',
 'Lat',
 'Lon',
 'Edu',
 'Degree',
 'Exp',
 'MinSalary',
 'MaxSalary',
 'MinHrlySalary',
 'MaxHrlySalary',
 'PayFrequency']

In [148]:
print(d1.count(), len(d1.columns))

1669891 53


In [149]:
d2= d1.select([c for c in d1.columns if c in columns])
print(d2.count(), len(d2.columns))

1669891 28


In [1]:
dff = d2.toPandas()
# print(dff.info())

In [151]:
sys.getsizeof(dff)

2353790399

In [161]:
d2= d1.filter(~((d1.Employer.contains('Government')) |
 (d1.Employer.contains('government')) |
 (d1.Employer.contains('Department ')) |
 (d1.Employer.contains('department ')) |
 (d1.Employer.contains('Army')) |
 (d1.Employer.contains('army')))
 )
print(d2.count(), len(d2.columns))

1551473 53


In [156]:
d2= d1.select([c for c in d1.columns if c in columns])
print(d2.count(), len(d2.columns))

1669891 28


In [157]:
d2.filter(col("CanonTitle").contains("Software")).groupBy('CanonTitle').count().sort(desc('count')).show()

+--------------------+-----+
|          CanonTitle|count|
+--------------------+-----+
|Software Developm...| 4395|
|  Software Developer| 1472|
|Principal Softwar...|  228|
|    Software Manager|  210|
|Software Developm...|  154|
|  Software Architect|  149|
|Software Engineer...|  143|
|Software Quality ...|   74|
|Software Test Eng...|   66|
|Software Systems ...|   60|
|     Software Tester|   52|
|Junior Software E...|   28|
|Junior Software D...|   27|
|Software Design E...|   25|
|Web Software Deve...|   18|
|Software Support ...|   16|
|C++ Software Engi...|   14|
|Software Account ...|   11|
|Software Developm...|   10|
|Embedded Software...|    3|
+--------------------+-----+
only showing top 20 rows



In [2]:
dffp = d2.toPandas()
# print(dffp.info())

In [165]:
dffp= dffp[columns]
dffp.shape

(1551473, 28)

In [166]:
sys.getsizeof(dffp)

2187312531

In [167]:
os.chdir(r"D:\OneDrive - Indian School of Business\Desktop\jobsAnalysis")

In [170]:
dff.to_csv(r"jobs_data_v2.csv", index=False)

In [169]:
%whos

Variable        Type                          Data/Info
-------------------------------------------------------
DataFrame       type                          <class 'pyspark.sql.dataframe.DataFrame'>
DateQuarter     type                          <class 'datequarter.quarter.DateQuarter'>
Parallel        type                          <class 'joblib.parallel.Parallel'>
SparkSession    type                          <class 'pyspark.sql.session.SparkSession'>
coalesce        function                      <function coalesce at 0x0000024239CB0790>
col             function                      <function col at 0x0000024239C95AF0>
columns         list                          n=28
count           function                      <function count at 0x0000024239CAB0D0>
countDistinct   function                      <function countDistinct at 0x0000024239CB0AF0>
csv             module                        <module 'csv' from 'c:\\U<...>\Anaconda3\\lib\\csv.py'>
d1              DataFrame               

In [171]:
del yr, df, dffp, d1, d2

# Read and filter skills

In [174]:
os.chdir(r'E:\DATA_JOBS\Skill')

In [176]:
df = spark.read.options(delimiter='\t').csv(r"{0}\\{1}".format(os.getcwd(), '2021/Skills_2021-01.txt'), header=True, inferSchema=True)
df.count(), len(df.columns)

(29769230, 9)

In [178]:
for yr in ['2019', '2020', '2021']:
    os.chdir(yr)
    for f in tqdm(os.listdir()):
        if f.endswith('.txt') and f!='Skills_2021-01.txt':
            print(f)
            d1=  spark.read.options(delimiter='\t').csv(r"{0}\\{1}".format(os.getcwd(), f), header=True, inferSchema=True)
            df= unionAll(df, d1)
            print(f, 'done')
            print(df.count(), len(df.columns))
    os.chdir('..')

  0%|          | 0/24 [00:00<?, ?it/s]

Skills_2019-01.txt
Skills_2019-01.txt done


  4%|▍         | 1/24 [00:15<06:07, 15.96s/it]

57143115 9
Skills_2019-02.txt
Skills_2019-02.txt done


 12%|█▎        | 3/24 [00:35<03:56, 11.24s/it]

...

1024435615 9
Skills_2021-11.txt
Skills_2021-11.txt done


 92%|█████████▏| 11/12 [16:05<01:41, 101.52s/it]

1064770685 9
Skills_2021-12.txt
Skills_2021-12.txt done


100%|██████████| 12/12 [17:48<00:00, 89.00s/it] 

1102718299 9


In [3]:
# dff.columns

In [195]:
os.chdir(r"D:\OneDrive - Indian School of Business\Desktop\jobsAnalysis")

In [196]:
sdf = spark.read.options(delimiter=',').csv(r"{0}\\{1}".format(os.getcwd(), 'jobs_data.csv'), header=True, inferSchema=True)
sdf.count(), len(sdf.columns)

(1669891, 28)

In [199]:
df= df.drop(df.JobDate)
df= df.drop(df.Salary)

print(df.count(), len(df.columns))

1102718299 7


In [204]:
del d1

In [203]:
sdf= sdf.select(col("JobID"),col("JobDate"))
print(sdf.count(), len(sdf.columns))

1669891 2


In [205]:
print(df.count(), len(df.columns))

1102718299 7


In [215]:
d1 = df.join(sdf, df.JobID == sdf.JobID, "inner")

In [216]:
print(d1.count(), len(d1.columns))

18515090 9


In [4]:
d= d1.toPandas()
# print(d.info())

In [224]:
d.columns=['JobID', 'Skill', 'SkillCluster', 'SkillClusterFamily',
       'IsSpecialized', 'IsBaseline', 'IsSoftware', 'drop1', 'drop2']

In [5]:
d= d[['JobID', 'Skill', 'SkillCluster', 'SkillClusterFamily',
       'IsSpecialized', 'IsBaseline', 'IsSoftware']]

In [228]:
dff.columns

Index(['JobID', 'JobDate', 'CleanTitle', 'CanonTitle', 'OccFam', 'OccFamName',
       'SOC', 'SOCName', 'ONET', 'ONETName', 'Employer', 'Sector',
       'SectorName', 'NAICS3', 'City', 'State', 'County', 'Lat', 'Lon', 'Edu',
       'Degree', 'Exp', 'MinSalary', 'MaxSalary', 'MinHrlySalary',
       'MaxHrlySalary', 'PayFrequency'],
      dtype='object')

In [229]:
d.columns

Index(['JobID', 'Skill', 'SkillCluster', 'SkillClusterFamily', 'IsSpecialized',
       'IsBaseline', 'IsSoftware'],
      dtype='object')

In [230]:
dff.to_csv(r"jobs_data.csv", index=False)
d.to_csv(r"skills_data.csv", index=False)

In [231]:
dff.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1669891 entries, 0 to 1669890
Data columns (total 27 columns):
 #   Column         Non-Null Count    Dtype         
---  ------         --------------    -----         
 0   JobID          1669891 non-null  int64         
 1   JobDate        1669891 non-null  datetime64[ns]
 2   CleanTitle     1669891 non-null  object        
 3   CanonTitle     1669891 non-null  object        
 4   OccFam         1669891 non-null  object        
 5   OccFamName     1669891 non-null  object        
 6   SOC            1669891 non-null  object        
 7   SOCName        1669891 non-null  object        
 8   ONET           1669891 non-null  object        
 9   ONETName       1669891 non-null  object        
 10  Employer       1669891 non-null  object        
 11  Sector         1669891 non-null  object        
 12  SectorName     1669891 non-null  object        
 13  NAICS3         1669891 non-null  int32         
 14  City           1669891 non-null  o

In [232]:
d.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18515090 entries, 0 to 18515089
Data columns (total 7 columns):
 #   Column              Non-Null Count     Dtype 
---  ------              --------------     ----- 
 0   JobID               18515090 non-null  int64 
 1   Skill               18515090 non-null  object
 2   SkillCluster        18515090 non-null  object
 3   SkillClusterFamily  18515090 non-null  object
 4   IsSpecialized       18515090 non-null  int32 
 5   IsBaseline          18515090 non-null  int32 
 6   IsSoftware          18515090 non-null  int32 
dtypes: int32(3), int64(1), object(3)
memory usage: 776.9+ MB


In [233]:
os.getcwd()

'D:\\OneDrive - Indian School of Business\\Desktop\\jobsAnalysis'